参考資料：
- [3. Model selection and evaluation](http://scikit-learn.org/stable/model_selection.html#model-selection)
- [3.1. Cross-validation: evaluating estimator performance](http://scikit-learn.org/stable/modules/cross_validation.html)

In [1]:
from IPython.display import Image

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

## 機械学習におけるモデル選択とは？

- モデルとモデルに付属するハイパーパラメータの同時最適化

## 機械学習における良いモデルとは？

- 良いモデル
    - テストデータに対する予測精度（汎化性能）が高いモデル
- 悪いモデル
    - テストデータに対する予測精度（汎化性能）が低いモデル
    - トレーニングデータを丸覚えしただけのモデルは、トレーニングデータに対する予測精度は最高だが、汎化性能が無いので悪いモデル

**要するに「良いモデル」を探すためには、トレーニングに用いていないデータで、学習済みモデルの評価を行う必要がある**

sklearnを使って、ラベル付きのデータをトレーニングデータとテストデータに分けよう。試しに、80%をトレーニングデータ、20%をテストデータにしてみよう。

In [3]:
import numpy as np
from sklearn import cross_validation
from sklearn import datasets
from sklearn import svm

In [4]:
iris = datasets.load_iris()

In [5]:
iris.data.shape, iris.target.shape

((150, 4), (150,))

In [6]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=3)

In [7]:
X_train.shape, y_train.shape

((120, 4), (120,))

In [8]:
X_test.shape, y_test.shape

((30, 4), (30,))

In [9]:
y_test

array([0, 0, 0, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 2, 0, 1, 2, 2, 0, 2, 2, 2,
       1, 0, 2, 2, 1, 1, 1])

In [10]:
model = svm.SVC(kernel='linear', C=1.0)

In [11]:
model

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [12]:
model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [13]:
# トレーニングスコア
model.score(X_train, y_train)

0.9916666666666667

In [14]:
# テストスコア(これがモデルの評価値)
model.score(X_test, y_test)

0.96666666666666667

## ハイパーパラメータ最適化とクロスバリデーション

モデルを作るときに

    model = svm.SVC(kernel='linear', C=1.0)

としたけけれども、この `kernel` や `C` って何？

- これが**モデルのハイパーパラメータ**と呼ばれるもの
- ハイパーパラメータはモデルによって全然違う
- あるモデル（例えばSVC）の汎化性能はこのハイパーパラメータの選択に大きく依存
- このハイパーパラメータの最適化にテストデータを使うと、テストデータに都合の良いハイパーパラメータが選ばれてしまうのでダメ（汎化性能が正確に計測できない...）
- そのために元のデータセットを、トレーニングセット、バリデーションセット、テストセットの３つに分割して、バリデーションセットでハイパーパラメータの選択を行うと良い（これで汎化性能が正確に測定できる）。

### データセット決め打ち３分割の問題点

- 問題点１：学習に用いることができるデータが大きく減ってしまう...
- 問題点２：最適化の結果選ばれるハイパーパラメータが、トレーニングセットとバリデーションセットとしてたまたま選ばれたデータに大きく依存してしまう...
- この問題を解くのが**クロスバリデーション (CV)**

## クロスバリデーション

- 元データからテストデータを切り分け、残りの部分を以下のように使ってクロスバリデーションを行う。これは k-fold クロスバリデーション (k=10) の例。
- テストデータは最後まで触っちゃダメ！モデル選択が終わった後、最終的なパフォーマンスを計測するときにだけ使う。

In [15]:
cv_url = 'http://i.imgur.com/N9HZktu.png'
Image(url=cv_url)
# Referece: https://github.com/bugra/pydata-dal-2015/blob/master/1.%20Scikit%20Learn%20-%20Introduction%20to%20Supervised%20Learning%20Problem%20and%20Cross%20Validation.ipynb

- 見ての通りクロスバリデーションは重い処理。
- なのでデータが腐るほどある時には、決め打ち３分割でもまぁ良い。注：この場合、３つのデータセットがそれぞれ十分に大きく、それぞれのデータセットが元のデータセットと十分に似ている必要がある。

## scikit-learnでCVをやってみよう

- 一番簡単な方法は [`cross_val_score`](http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.cross_val_score.html#sklearn.cross_validation.cross_val_score) を用いる方法。
- 計算されるscoreはデフォルトではモデルが持っているscore関数を用いる。変更したい場合は`scoring`パラメータで設定する
- cvパラメータが整数の時には [KFold](http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.KFold.html#sklearn.cross_validation.KFold) か [StratifiedKFold](http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedKFold.html#sklearn.cross_validation.StratifiedKFold) が使われる。ちなみに StratifiedKFold が使われるのは、モデルが [ClassifierMixin](http://scikit-learn.org/stable/modules/generated/sklearn.base.ClassifierMixin.html#sklearn.base.ClassifierMixin) から派生している場合。
- cvにcross-validation iteratorを渡しても良い

In [16]:
model = svm.SVC(kernel='linear', C=1)
scores = cross_validation.cross_val_score(model, X_train, y_train, cv=5)

In [17]:
scores

array([ 0.95833333,  0.95833333,  1.        ,  1.        ,  0.95833333])

95%の信頼区間を計算。

In [18]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 1.96))

Accuracy: 0.97 (+/- 0.04)


In [19]:
from sklearn import metrics
scores = cross_validation.cross_val_score(model, X_train, y_train, cv=5, 
                                          scoring='f1_weighted')
scores

array([ 0.95816993,  0.95816993,  1.        ,  1.        ,  0.95816993])

In [20]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 1.96))

Accuracy: 0.97 (+/- 0.04)


scoringパラメータについての詳細は以下のリンクを参照
- [The scoring parameter: defining model evaluation rules](http://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)

In [21]:
# cross_val_scoreのcvパラメータにcross-validation iteratorを渡す場合

n_samples = X_train.shape[0]
cv = cross_validation.ShuffleSplit(n_samples, n_iter=10, 
                                   test_size=0.3, random_state=0)

cross_validation.cross_val_score(model, X_train, y_train, cv=cv)

array([ 0.88888889,  0.97222222,  1.        ,  1.        ,  0.94444444,
        1.        ,  0.97222222,  1.        ,  1.        ,  1.        ])

## データの前処理とテストデータ

- データの前処理がデータ依存な場合（例：ホワイトニング）、トレーニングデータでそれらの前処理を学習し、学習済みの前処理をテストデータに施すこと。
- 参考：[Dataset transformations](http://scikit-learn.org/stable/data_transforms.html#data-transforms)

In [22]:
from sklearn import preprocessing
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    iris.data, iris.target, test_size=0.2, random_state = 4)

In [23]:
scaler = preprocessing.StandardScaler()

In [24]:
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [25]:
X_train_transformed = scaler.transform(X_train)

In [26]:
model = svm.SVC(C=1).fit(X_train_transformed, y_train)

In [27]:
print "Preprocessing training data"

print "\n---- Before preprocessing ----"
print("mean: %s" % X_train.mean(axis=0))
print("std: %s" % X_train.std(axis=0))

print "\n---- After preprocessing ----"
print("mean: %s" % X_train_transformed.mean(axis=0))
print("std: %s" % X_train_transformed.std(axis=0))

Preprocessing training data

---- Before preprocessing ----
mean: [ 5.88916667  3.04916667  3.89166667  1.26416667]
std: [ 0.81637367  0.41432593  1.68935803  0.74294412]

---- After preprocessing ----
mean: [ -4.81096644e-16  -3.50645439e-16   5.18104078e-16  -9.41839199e-16]
std: [ 1.  1.  1.  1.]


In [28]:
# ここが重要！
# トレーニングデータでフィッティングした前処理をテストデータに施す
X_test_transformed = scaler.transform(X_test)

In [29]:
print "Preprocessing test data"

print "\n---- Before preprocessing ----"
print("mean: %s" % X_test.mean(axis=0))
print("std: %s" % X_test.std(axis=0))

print "\n---- After preprocessing ----"
print("mean: %s" % X_test_transformed.mean(axis=0))
print("std: %s" % X_test_transformed.std(axis=0))

Preprocessing test data

---- Before preprocessing ----
mean: [ 5.66        3.07333333  3.22666667  0.93666667]
std: [ 0.83530434  0.49661074  1.92161969  0.77394372]

---- After preprocessing ----
mean: [-0.28071296  0.05832767 -0.39364065 -0.44081377]
std: [ 1.02318872  1.19859923  1.13748516  1.04172535]


In [30]:
model.score(X_test_transformed, y_test)

0.93333333333333335

[Pipeline](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) を用いると、CVする時の処理を簡単にできる。

In [31]:
from sklearn.pipeline import make_pipeline

In [32]:
model = make_pipeline(preprocessing.StandardScaler(), svm.SVC(C=1))

In [33]:
cross_validation.cross_val_score(model, X_train, y_train, cv=3)

array([ 0.92682927,  0.975     ,  1.        ])

## [CVで予測値を得る方法](http://scikit-learn.org/stable/modules/cross_validation.html#obtaining-predictions-by-cross-validation)

[`cross_val_predict`](http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.cross_val_predict.html#sklearn.cross_validation.cross_val_predict)を用いる

In [34]:
predicted = cross_validation.cross_val_predict(model, 
                                               X_train, y_train, cv=10)

In [35]:
y_train.shape, predicted.shape

((120,), (120,))

In [36]:
metrics.accuracy_score(y_train, predicted)

0.96666666666666667

## [CVイテレータ](http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators)

CV用のインデックスを生成するイテレータの作り方

- K-fold (KFold)
- Stratified k-fold (StratifiedKFold)
- Leave-One-Out - LOO (LeaveOneOut)
- Leave-P-Out - LPO (LeavePOut)
- Leave-One-Label-Out - LOLO (LeaveOneLabelOut)
- Leave-P-Label-Out (LeavePLabelOut)
- Shuffle & Split (ShuffleSplit)
- Predefined Split (PredefinedSplit)

### k-Fold

In [37]:
import numpy as np
from sklearn.cross_validation import KFold

In [38]:
kf = KFold(4, n_folds=4)

In [39]:
for train, test in kf:
    print("%s %s" % (train, test))

[1 2 3] [0]
[0 2 3] [1]
[0 1 3] [2]
[0 1 2] [3]


### Stratified k-fold

In [40]:
from sklearn.cross_validation import StratifiedKFold

In [41]:
labels = [0, 0, 1, 1, 1, 1]

In [42]:
skf = StratifiedKFold(labels, 2)

In [43]:
for train, test in skf:
    print("%s %s" % (train, test))

[1 4 5] [0 2 3]
[0 2 3] [1 4 5]


### Leave-One-Out (LOO)

In [44]:
from sklearn.cross_validation import LeaveOneOut

In [45]:
loo = LeaveOneOut(4)

In [46]:
for train, test in loo:
    print("%s %s" % (train, test))

[1 2 3] [0]
[0 2 3] [1]
[0 1 3] [2]
[0 1 2] [3]


### Random permutations cross-validation a.k.a. Shuffle & Split

[ShuffleSplit](http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.ShuffleSplit.html#sklearn.cross_validation.ShuffleSplit)

In [47]:
ss = cross_validation.ShuffleSplit(5, n_iter=3, test_size=0.4,
                                   random_state=0)

In [48]:
for train_index, test_index in ss:
    print("%s %s" % (train_index, test_index))

[1 3 4] [2 0]
[1 4 3] [0 2]
[4 0 2] [1 3]
